In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from math import floor
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

In [27]:
df = pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\old\data\\1h\LTCUSDT_1h.csv", index_col='Time')
df.index = pd.to_datetime(df.index)

In [28]:
# williams %R
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max()
    lowl = low.rolling(lookback).min()
    wr = -100 * ((highh - close) / (highh - lowl))
    return wr

df['wr_14'] = get_wr(df['High'], df['Low'], df['Close'], 14)



In [29]:
def get_macd(price, fast, slow, smooth):
    exp1 = price.ewm(span = fast, adjust=False).mean()
    exp2 = price.ewm(span = slow, adjust=False).mean()
    macd = pd.DataFrame(exp1-exp2).rename(columns={'Close':'macd'})
    
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    return macd, signal, hist
    
get_macd(df['Close'], 26, 12, 9)[1]

,signal
Time,
2022-03-14 18:00:00,0.000000
2022-03-14 19:00:00,-0.011168
2022-03-14 20:00:00,-0.027130
2022-03-14 21:00:00,-0.057961
2022-03-14 22:00:00,-0.120295
...,...
2022-04-25 03:00:00,0.558295
2022-04-25 04:00:00,0.635713
2022-04-25 05:00:00,0.718259


In [30]:
df['macd'] = get_macd(df['Close'], 26, 12, 9)[0]
df['macd_signal'] = get_macd(df['Close'], 26, 12, 9)[1]
df['macd_hist'] = get_macd(df['Close'], 26, 12, 9)[2]
df = df.dropna()

In [31]:
def implement_wr_macd_strategy(prices, wr, macd, macd_signal):    
    buy_price = []
    sell_price = []
    wr_macd_signal = []
    signal = 0

    for i in range(len(wr)):
        if wr[i-1] > -50 and wr[i] < -50 and macd[i] > macd_signal[i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                wr_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                wr_macd_signal.append(0)
                
        elif wr[i-1] < -50 and wr[i] > -50 and macd[i] < macd_signal[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                wr_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                wr_macd_signal.append(0)
        
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            wr_macd_signal.append(0)
            
    return buy_price, sell_price, wr_macd_signal
            
buy_price, sell_price, wr_macd_signal = implement_wr_macd_strategy(df['Close'], df['wr_14'], df['macd'], df['macd_signal'])

In [32]:
# POSITION

position = []
for i in range(len(wr_macd_signal)):
    if wr_macd_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)
        
for i in range(len(df['Close'])):
    if wr_macd_signal[i] == 1:
        position[i] = 1
    elif wr_macd_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]
        
close_price = df['Close']
wr = df['wr_14']
macd_line = df['macd']
signal_line = df['macd_signal']
wr_macd_signal = pd.DataFrame(wr_macd_signal).rename(columns = {0:'wr_macd_signal'}).set_index(df.index)
position = pd.DataFrame(position).rename(columns = {0:'wr_macd_position'}).set_index(df.index)

frames = [close_price, wr, macd_line, signal_line, wr_macd_signal, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

strategy.head()

,Close,wr_14,macd,macd_signal,wr_macd_signal,wr_macd_position
Time,,,,,,
2022-03-15 07:00:00,103.4,-64.583333,-0.668970,-0.709238,0,1
2022-03-15 08:00:00,102.8,-78.723404,-0.497673,-0.666925,0,1
2022-03-15 09:00:00,103.3,-78.048780,-0.397680,-0.613076,0,1
2022-03-15 10:00:00,103.8,-65.853659,-0.354692,-0.561399,0,1
2022-03-15 11:00:00,103.9,-63.414634,-0.324947,-0.514109,0,1


In [33]:
# BACKTESTING

df_ret = pd.DataFrame(np.diff(df['Close'])).rename(columns = {0:'returns'})
wr_macd_strategy_ret = []

for i in range(len(df_ret)):
    try:
        returns = df_ret['returns'][i] * strategy['wr_macd_position'][i]
        wr_macd_strategy_ret.append(returns)
    except:
        pass
    
wr_macd_strategy_ret_df = pd.DataFrame(wr_macd_strategy_ret).rename(columns = {0:'wr_macd_returns'})

investment_value = 100000
number_of_stocks = floor(investment_value / df['Close'][0])
wr_macd_investment_ret = []

for i in range(len(wr_macd_strategy_ret_df['wr_macd_returns'])):
    returns = number_of_stocks * wr_macd_strategy_ret_df['wr_macd_returns'][i]
    wr_macd_investment_ret.append(returns)

wr_macd_investment_ret_df = pd.DataFrame(wr_macd_investment_ret).rename(columns = {0:'investment_returns'})
total_investment_ret = round(sum(wr_macd_investment_ret_df['investment_returns']), 2)
profit_percentage = floor((total_investment_ret / investment_value) * 100)
print(cl('Profit gained from the W%R MACD strategy by investing $100k in df : {}'.format(total_investment_ret), attrs = ['bold']))
print(cl('Profit percentage of the W%R MACD strategy : {}%'.format(profit_percentage), attrs = ['bold']))

Profit gained from the W%R MACD strategy by investing $100k in df : -8896.4
Profit percentage of the W%R MACD strategy : -9%
